In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import ROOT
import numba

Welcome to JupyROOT 6.18/04


In [2]:
!ls output/tuples-v2-training-v2-t1/testing/

e_DY.h5    jet_DY.h5  jet_TT.h5  mu_DY.h5   tau_HTT.h5
e_W.h5     jet_QCD.h5 jet_W.h5   mu_W.h5


In [6]:
@numba.njit
def calc_weights(tau_pt, bin_weights, bin_edges):
    n_tau = tau_pt.shape[0]
    n_bins = len(bin_edges)
    weights = np.zeros(n_tau)
    for n in range(n_tau):
        k = 0
        while bin_edges[k + 1] <= tau_pt[n]:
            k += 1
        weights[n] = bin_weights[k]
    return weights

In [7]:
def CreateWeightDataFrame(input_file, tau_type, weight_file, output_file):
    df = pandas.read_hdf(input_file, 'taus', columns=['tau_pt'])
    df['tau_pt'] = pandas.Series(df.tau_pt *(1000 - 20) + 20, index=df.index)
    
    bins=np.arange(20, 200, step=5)
    bins = np.append(bins, np.arange(200, 300, step=10))
    bins = np.append(bins, np.arange(300, 400, step=50))
    bins = np.append(bins, [400, 1000])
    
    cnt, bin_edges = np.histogram(df.tau_pt, bins=bins)
    idx = np.argmin(cnt > 100)
    if cnt[idx] <= 100:
        raise RuntimeError('bins {} has too few entries ({})'.format(bin_edges[idx], cnt[idx]))
    
    wfile = ROOT.TFile(weight_file, 'OPEN')
    tau_spec_hist = wfile.Get(tau_type + '_pt')
    
    tau_spec = np.zeros(len(bins) - 1)
    for n in range(len(tau_spec)):
        pt_bin_min = tau_spec_hist.FindBin(bins[n])
        pt_bin_max = tau_spec_hist.FindBin(bins[n+1]) - 1
        tau_spec[n] = tau_spec_hist.Integral(pt_bin_min, pt_bin_max)
        
    tau_spec_norm = np.sum(tau_spec)
    cnt_norm = np.sum(cnt)
    print('Total events: n_evt_dataset={}, n_evt_shape={}'.format(cnt_norm, tau_spec_norm))
    
    bin_weights = tau_spec / cnt * (cnt_norm / tau_spec_norm)
    print('Bin weights:', bin_weights)
    
    weights = calc_weights(df.tau_pt.values, bin_weights, bin_edges)
    
    if np.count_nonzero(np.isnan(weights)) != 0:
        raise RuntimeError("NaN in weights")
        
    print('Weights: mean={}, stddev={}, min={}, max={}' \
          .format(np.mean(weights), np.std(weights), np.amin(weights), np.amax(weights)))
        
    df_weights = pandas.DataFrame(data={'weight': weights})
    df_weights.to_hdf(output_file, key='weight', mode='w', format='fixed', complevel=9)
    
    wfile.Close()
    
    return df_weights, bin_weights

In [8]:
CreateWeightDataFrame('output/tuples-v2-training-v2-t1/testing/jet_W.h5', 'jet', 'W_jet_pt.root',
                      'output/spectrum_weights/jet_W_weights.h5');

Total events: n_evt_dataset=871014, n_evt_shape=930645.0
Bin weights: [5.11781385 2.43931325 1.25863205 0.68526095 0.41318753 0.26051475
 0.19551723 0.16335133 0.18074713 0.16897884 0.14298231 0.13353017
 0.12319022 0.11081819 0.10958594 0.10694071 0.09402986 0.09238231
 0.09471431 0.08788029 0.08308524 0.0858205  0.08236764 0.07481439
 0.08126701 0.07462265 0.08010377 0.07898102 0.07436276 0.07227936
 0.06987676 0.07480896 0.06233729 0.06784958 0.07522802 0.08040162
 0.07147338 0.07574463 0.06880196 0.07058653 0.05992203 0.08164453
 0.06253877 0.06980657 0.04972102 0.05399568 0.04959972 0.05988146
 0.06400966]
Weights: mean=0.9999999999999998, stddev=1.474104713269003, min=0.049599716444615374, max=5.117813846445207


In [9]:
CreateWeightDataFrame('output/tuples-v2-training-v2-t1/testing/jet_TT.h5', 'jet', 'pt_spec.root',
                      'output/spectrum_weights/jet_TT_weights.h5');

Total events: n_evt_dataset=1606307, n_evt_shape=21296327.0
Bin weights: [4.32770465 3.62700961 2.85568522 2.12585011 1.59180517 1.20766187
 0.91094918 0.69243395 0.98364794 0.92188564 0.61093735 0.58248799
 0.39276624 0.38135675 0.25352988 0.24486762 0.27716974 0.27241703
 0.26652437 0.26159807 0.13012005 0.12510956 0.12197671 0.12426431
 0.06453833 0.06359397 0.0621864  0.06155588 0.03386413 0.03389881
 0.03329203 0.03299298 0.0193986  0.01996341 0.02028936 0.0200535
 0.01537409 0.01533402 0.01324563 0.01156491 0.01162798 0.01096086
 0.01081285 0.01043066 0.01100079 0.01071506 0.00993271 0.00929333
 0.00670711]
Weights: mean=0.9999999999999996, stddev=1.249745753591426, min=0.006707108895412067, max=4.32770464827045


In [10]:
CreateWeightDataFrame('output/tuples-v2-training-v2-t1/testing/tau_HTT.h5', 'tau', 'pt_spec.root',
                      'output/spectrum_weights/tau_HTT_weights.h5');

Total events: n_evt_dataset=827370, n_evt_shape=1633147.0
Bin weights: [1.24216549 1.33940311 1.37508118 1.32127911 1.18988325 1.02091586
 0.82971252 0.6569235  0.71503662 0.71382414 0.51971381 0.52124434
 0.5145944  0.51792998 0.51280429 0.51476234 0.51786674 0.52559285
 0.50342724 0.51047299 0.51356396 0.49521379 0.52031816 0.52335561
 0.50303814 0.47427399 0.52321327 0.52380353 0.48985274 0.51314309
 0.49180729 0.54308684 0.53439964 0.43516574 0.52799812 0.46753343
 0.50573132 0.52010664 0.56902996 0.50063315 0.53874662 0.51578028
 0.56728282 0.44281542 0.46023099 0.45898078 0.52693784 0.50323345
 0.54652565]
Weights: mean=1.0, stddev=0.315269884094354, min=0.4351657354693823, max=1.375081181761348


In [11]:
CreateWeightDataFrame('output/tuples-v2-training-v2-t1/testing/e_DY.h5', 'e', 'pt_spec.root',
                      'output/spectrum_weights/e_DY_weights.h5');

Total events: n_evt_dataset=1120797, n_evt_shape=12583771.0
Bin weights: [1.55592572 1.69013167 1.79464495 1.94742802 2.3042488  2.09875262
 0.94535902 0.50580069 0.51635435 0.50635745 0.24832196 0.24322014
 0.13679193 0.13709543 0.08977887 0.09068681 0.09385533 0.09408982
 0.09428729 0.09417997 0.08766884 0.08793932 0.08749235 0.08809928
 0.08767431 0.08819318 0.08829791 0.08711007 0.08731248 0.08788315
 0.08786686 0.08810478 0.08763098 0.08810224 0.0870017  0.08803332
 0.08750489 0.0875139  0.08783912 0.08846097 0.08763708 0.08815075
 0.08800654 0.0874686  0.08584758 0.08650115 0.08707654 0.08543148
 0.08642863]
Weights: mean=1.0, stddev=0.8239416551131274, min=0.08543148025130698, max=2.3042487968829057


In [12]:
CreateWeightDataFrame('output/tuples-v2-training-v2-t1/testing/mu_DY.h5', 'mu', 'pt_spec.root',
                      'output/spectrum_weights/mu_DY_weights.h5');

Total events: n_evt_dataset=1128400, n_evt_shape=12827880.0
Bin weights: [1.57022402 1.6911952  1.78519423 1.91731604 2.2371885  2.22143843
 0.99102406 0.50892939 0.51607323 0.51403013 0.24987281 0.25143594
 0.1382878  0.13907554 0.09015367 0.09041618 0.09395495 0.0939681
 0.09417471 0.09400188 0.08532811 0.08486144 0.08519449 0.08545496
 0.08537942 0.08551251 0.0852023  0.08586437 0.08556176 0.0849507
 0.0848176  0.08596066 0.08427775 0.08586813 0.08626311 0.08658841
 0.08682871 0.08723305 0.08685519 0.08646738 0.08647106 0.085529
 0.08638174 0.08662373 0.08635652 0.0866376  0.08557564 0.0864821
 0.08339901]
Weights: mean=1.0, stddev=0.826584160475319, min=0.08339901261561201, max=2.2371885007499293
